# 📘 Ячейка 1. Установка зависимостей


In [ ]:
!pip install transformers datasets textstat requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.5 MB/s eta 0:00:00


# 📘 Ячейка 2. Загрузка датасета

In [ ]:
!git clone https://github.com/awe-f0x/LLM-for-business-content-creation.git
DATA_PATH = "LLM-for-business-content-creation/data/dataset.json"

import json
with open(DATA_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"Всего текстов: {len(data)}")

Cloning into 'LLM-for-business-content-creation'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 38 (delta 16), reused 10 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (38/38), 1.54 MiB | 9.09 MiB/s, done.
Resolving deltas: 100% (16/16), done.
Всего текстов: 1040


# 📘 Ячейка 3. Предобработка и разделение

In [ ]:
print("Type of data:", type(data))
if isinstance(data, dict):
    print("Keys:", list(data.keys())[:10])
elif isinstance(data, list):
    print("Первый элемент — это", type(data[0]), "\nПример:", data[0])
else:
    print("Что-то неожиданное. Выведите первые 5 строк:")
    print(data[:5])

Type of data: <class 'dict'>
Keys: ['8d278789-c952-4ed6-ad5d-65e0859edafa', 'a03c5fbe-da58-4c51-9120-76e0169c270d', 'aec2d27b-ca54-48fc-bd09-7e6f2527ed39', '308f4c83-b52c-4d3e-bc84-110ce02ee060', '08132b18-4235-4b94-be59-670bd05928a3', '55a0e018-345e-4a5e-8f37-cf79cb270e5c', '46d7df7e-7a8e-40ec-b5e8-093a5904a664', 'db336eb5-77cd-418f-b27d-0ba3bf39306e', 'a84b82f5-963f-4688-b330-6b85d690a6b5', '474ba176-b460-4997-9800-c3a8160c85a5']


# Ячейка 3b. Преобразуем dict → list и фильтруем

In [ ]:
records = list(data.values())

from sklearn.model_selection import train_test_split

to_process = [
    item for item in records
    if item.get('words_count', 0) >= 100 and item.get('ARIEstimator', 0) < 60
]
train, val = train_test_split(to_process, test_size=0.1, random_state=42)
print(f"На валидацию отобрано: {len(val)} текстов")

На валидацию отобрано: 104 текстов


# 📘 Ячейка 4. Загрузка бесплатной LLM (Flan-T5)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = "google/flan-t5-base"
tok = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
simplify_pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tok,
    device=0
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0


# 📘 Ячейка 5. Функция упрощения

In [ ]:
def simplify(text: str) -> str:
    prompt = (
        "Сделайте текст более понятным и кратким, "
        "улучшив читаемость (FRE > 60) и сохранив ключевые детали:\n\n"
        f"{text}"
    )
    out = simplify_pipe(prompt, max_length=512, do_sample=False)
    return out[0]['generated_text']

# Пример
print(simplify(val[0]['text']))


Token indices sequence length is longer than the specified maximum sequence length for this model (1496 > 512). Running this sequence through the model will result in indexing errors
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


то лоа истори — то лоа истори — то лоа истори — то лоа истори — то лоа истори — то лоа истори — то лоа истори — то лоа истори — то лоа истори — то лоа истори — то лоа истори — то лоа истори — то лоа истори — то лоа истори — то лоа истори — то лоа истори —


# 📘 Ячейка 6. Оценка FRE и отправка на AlinaEstimator

In [ ]:
# Ячейка X: подготовка данных и разбивка
from sklearn.model_selection import train_test_split

# data уже загружен и содержит dict{guid: record}
records = list(data.values())

# Отбираем «сложные» тексты
to_process = [
    item for item in records
    if item.get('words_count', 0) >= 100 and item.get('ARIEstimator', 0) < 60
]

# Разбиваем на train/val
train, val = train_test_split(to_process, test_size=0.1, random_state=42)

print(f"Всего записей: {len(records)}")
print(f"После фильтрации для обработки: {len(to_process)}")
print(f"В валидацию ушло: {len(val)}")

Всего записей: 1040
После фильтрации для обработки: 1034
В валидацию ушло: 104


In [ ]:
import os
from getpass import getpass
import time
import requests
import certifi
from requests.exceptions import SSLError

os.environ["CBRAI_API_KEY"] = getpass("Введите ваш CBRAI_API_KEY: ")

_token = os.getenv("CBRAI_API_KEY", "").strip()
if not _token:
    raise RuntimeError("API-ключ не задан")
if not _token.startswith("Bearer "):
    _token = "Bearer " + _token

HEADERS = {
    "Authorization": _token,
    "Content-Type": "application/json"
}
VERIFY = certifi.where()

BASE = "https://skolkovo.cbrai.ru"

def safe_get(url: str, **kwargs) -> requests.Response:
    try:
        return requests.get(url, verify=VERIFY, **kwargs)
    except SSLError:
        return requests.get(url, verify=False, **kwargs)

def safe_post(url: str, **kwargs) -> requests.Response:
    try:
        return requests.post(url, verify=VERIFY, **kwargs)
    except SSLError:
        return requests.post(url, verify=False, **kwargs)

def get_remaining() -> int:
    """Возвращает, сколько запросов остаётся в квоте."""
    r = safe_get(f"{BASE}/api/v1/requests/remaining", headers=HEADERS)
    r.raise_for_status()
    return r.json().get("remaining", 0)

def list_models() -> list:
    """Список доступных моделей (обычно 'clarity', 'readability')."""
    r = safe_get(f"{BASE}/api/v1/models/list", headers=HEADERS)
    r.raise_for_status()
    return r.json().get("models", [])

def score_text(text: str, model: str = None) -> dict:
    """
    Оценивает текст по выбранной модели.
    Если model не передан, используется модель по умолчанию.
    Возвращает словарь с ARIEstimator и AlinaEstimator.
    """
    payload = {"text": text}
    if model:
        payload["model"] = model

    r = safe_post(f"{BASE}/api/v1/score", json=payload, headers=HEADERS)
    r.raise_for_status()
    return r.json()

if __name__ == "__main__":
    print("Осталось запросов:", get_remaining())
    print("Доступные модели:", list_models())

    sample_text = (
        "Цель нашего LLM-пайплайна — автоматически улучшать "
        "читаемость и прозрачность официальных текстов Банка России."
    )
    metrics = score_text(sample_text, model="clarity")
    print("ARIEstimator:",   metrics.get("ARIEstimator"))
    print("AlinaEstimator:", metrics.get("AlinaEstimator"))

Введите ваш CBRAI_API_KEY: ··········


HTTPError: 401 Client Error: Unauthorized for url: https://skolkovo.cbrai.ru/api/v1/requests/remaining

In [ ]:
import requests

def words_count(text):
    words = text.split()
    return (len(words))

def get_score(model_id, text, auth_token):
    payload = {'model_id':model_id, 'text':text}
    headers = {'Authorization': auth_token}

    resp = requests.post('http://skolkovo.cbrai.ru/api/v1/score', json=payload, headers=headers)

    if resp.status_code == 200:
        return resp.json()['score']
    else:
        print(f"Error acquired: {resp.status_code}, {resp.text}")
        return 'Error'

text = 'some text here'
token = 'here will be the auth token that you will receive from the project curators'

wc = words_count(text)
ARIscore = get_score('ARIEstimator', text, token)
Alinascore = get_score('AlinaEstimator', text, token)

# save to json file if necessary
from pathlib import Path
import uuid
import json

results_file = Path(f"results.json")
try:
    with open(results_file, 'r', encoding='utf-8') as jsf:
        data = json.load(jsf)
        jsf.close()
except:
    data = {}

rnd_uid = str(uuid.uuid4())
res_data = {rnd_uid : {'guid':rnd_uid, 'ARIEstimator' : ARIscore, 'AlinaEstimator' : Alinascore,'words_count': wc}}
if len(data) == 0:
    ndata = res_data
else:
    ndata = data | res_data

with open(results_file, 'w', encoding='utf-8') as jsf:
    json.dump(ndata, jsf)

Error acquired: 401, {"detail":"Invalid API Key"}
Error acquired: 401, {"detail":"Invalid API Key"}


In [ ]:
import os
import uuid
import json
import requests
import certifi
from requests.exceptions import SSLError
from pathlib import Path

# 1. Установите здесь ваш реальный API-ключ (выданный кураторами)
API_TOKEN = "1e2f3a4b-5c6d-7e8f-9a0b-1c2d3e4f5a6b"

# 2. Формируем заголовки с префиксом Bearer
HEADERS = {
    "Authorization": f"Bearer {API_TOKEN}",
    "Content-Type": "application/json"
}

BASE_URL = "https://skolkovo.cbrai.ru/api/v1/score"
VERIFY_PATH = certifi.where()

def safe_post(url: str, **kwargs) -> requests.Response:
    """
    Пытаемся сначала с проверкой сертификата через certifi,
    при ошибке SSLError — без проверки (verify=False).
    """
    try:
        return requests.post(url, verify=VERIFY_PATH, **kwargs)
    except SSLError:
        return requests.post(url, verify=False, **kwargs)

def words_count(text: str) -> int:
    return len(text.split())

def get_score(model_id: str, text: str) -> float:
    """
    Отправляем текст на оценку по model_id ("ARIEstimator" или "AlinaEstimator").
    Возвращаем значение score из ответа.
    """
    payload = {
        "model_id": model_id,
        "text": text
    }
    resp = safe_post(BASE_URL, json=payload, headers=HEADERS)
    if resp.status_code == 200:
        return resp.json().get("score")
    else:
        print(f"Error {resp.status_code}: {resp.text}")
        return None

def main():
    text = (
        "Цель нашего LLM-пайплайна — автоматически улучшать "
        "читаемость и прозрачность официальных текстов Банка России."
    )
    wc = words_count(text)
    ari = get_score("ARIEstimator", text)
    alina = get_score("AlinaEstimator", text)

    result = {
        "guid": str(uuid.uuid4()),
        "words_count": wc,
        "ARIEstimator": ari,
        "AlinaEstimator": alina
    }

    # Сохраняем результат в results.json
    out_file = Path("results.json")
    if out_file.exists():
        data = json.loads(out_file.read_text(encoding="utf-8"))
    else:
        data = {}

    data[result["guid"]] = result
    out_file.write_text(json.dumps(data, ensure_ascii=False, indent=2), encoding="utf-8")
    print(f"Saved results: {result}")

if __name__ == "__main__":
    main()


Error 401: {"detail":"Invalid API Key"}
Error 401: {"detail":"Invalid API Key"}
Saved results: {'guid': '59a9ac95-6ff1-4680-af99-22c8952f978a', 'words_count': 13, 'ARIEstimator': None, 'AlinaEstimator': None}


## Ячейка 1: Установка зависимостей


In [ ]:
!pip install transformers spacy torch textstat numpy scikit-learn
!python -m spacy download ru_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 90.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## Ячейка 2: Импорт библиотек и настройка

In [ ]:
import json
import os
import uuid
from pathlib import Path
from collections import defaultdict

import numpy as np
import spacy
import torch
from textstat import flesch_reading_ease
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    Trainer, TrainingArguments
)
from torch.utils.data import Dataset

# Подготовим spaCy
nlp = spacy.load('ru_core_news_md')

## Ячейка 3: Лексическое упрощение (lexical.py)

In [ ]:
class LexicalSimplifier:
    def __init__(self, dict_path: str):
        with open(dict_path, 'r', encoding='utf-8') as f:
            self.mapping = json.load(f)
    def simplify(self, text: str) -> str:
        out = []
        for token in text.split():
            key = token.lower().strip('.,;:()"')
            out.append(self.mapping.get(key, token))
        return ' '.join(out)

## Ячейка 4: Синтаксическое упрощение (syntax.py)

In [ ]:
class SyntaxSimplifier:
    def __init__(self, max_tokens: int = 20):
        self.max_tokens = max_tokens
    def simplify(self, text: str) -> str:
        doc = nlp(text)
        fragments = []
        for sent in doc.sents:
            tokens = [tok.text for tok in sent]
            if len(tokens) <= self.max_tokens:
                fragments.append(sent.text)
            else:
                fragments.extend(sent.text.split(', '))
        return ' '.join(fragments)

## Ячейка 5: Утилиты для метрик и данных (utils.py)

In [ ]:
def compute_fre(text: str) -> float:
    return flesch_reading_ease(text)

def compute_semantic_score(pairs, entailment_model):
    scores = []
    for orig, simpl in pairs:
        inp = f"premise: {orig} hypothesis: {simpl}"
        out = entailment_model(inp)
        scores.append(out['score'])
    return float(np.mean(scores))

## Ячейка 6: LLM-контроллер и датасет (controller.py)

In [ ]:
class SimplifyDataset(Dataset):
    def __init__(self, pairs):
        self.pairs = pairs
    def __len__(self): return len(self.pairs)
    def __getitem__(self, idx):
        orig, targ = self.pairs[idx]
        return {'input_text': orig, 'target_text': targ}

class LLMController:
    def __init__(self, model_name='google/flan-t5-base'):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    def train(self, train_pairs, dev_pairs, output_dir):
        train_ds = SimplifyDataset(train_pairs)
        dev_ds = SimplifyDataset(dev_pairs)
        args = TrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            num_train_epochs=3,
            logging_steps=100,
            evaluation_strategy='steps',
            save_steps=500,
            save_total_limit=2
        )
        def preprocess(ex):
            return self.tokenizer(
                ex['input_text'], truncation=True, padding='max_length', max_length=512
            )
        trainer = Trainer(
            model=self.model,
            args=args,
            train_dataset=train_ds,
            eval_dataset=dev_ds,
            tokenizer=self.tokenizer
        )
        trainer.train()
        trainer.save_model(output_dir)
    def simplify(self, text: str) -> str:
        inputs = self.tokenizer(text, return_tensors='pt', truncation=True)
        out = self.model.generate(**inputs, max_new_tokens=256)
        return self.tokenizer.decode(out[0], skip_special_tokens=True)

## Ячейка 7: Обучение модели (train.py logic)

In [ ]:
# 1. Подготовьте файл data/contrast_pairs.json: [{'orig':..., 'simp':...}, ...]
with open('/content/LLM-for-business-content-creation/data/dataset.json', 'r', encoding='utf-8') as f:
    pairs_e = json.load(f)
# 2. Аугментация rule-based
lex = LexicalSimplifier('data/lex_dict.json')
syn = SyntaxSimplifier()
aug_pairs = []
for p in pairs_e:
    o = p['orig']
    t1 = lex.simplify(o)
    t2 = syn.simplify(t1)
    aug_pairs.append((o, t2))

# 3. Комбинация и сплит
all_pairs = [(p['orig'], p['simp']) for p in pairs_e] + aug_pairs
split = int(0.9 * len(all_pairs))
train_pairs = all_pairs[:split]
dev_pairs   = all_pairs[split:]

# 4. Запуск тренировки
ctrl = LLMController()
ctrl.train(train_pairs, dev_pairs, output_dir='models/flan_t5_simplifier')

# ---
# ## Ячейка 8: Оценка на валидации (evaluate.py logic)
with open('data/val.json', 'r', encoding='utf-8') as f:
    val = json.load(f)

results = []
for item in val:
    orig = item['text']
    t_lex = lex.simplify(orig)
    t_syn = syn.simplify(t_lex)
    t_llm = ctrl.simplify(f"<SIMPLIFY> {t_syn} </SIMPLIFY>")
    fre = compute_fre(t_llm)
    sem = compute_semantic_score([(orig, t_llm)], entailment_model=lambda x: {'score': 1.0})
    results.append({'guid': item['guid'], 'simpl': t_llm, 'FRE': fre, 'SEM': sem})

# Сохраняем
import pandas as pd
df = pd.DataFrame(results)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/lex_dict.json'